<a href="https://colab.research.google.com/github/KLN-TBN/Notebooks/blob/main/BackwardAndForwardPassBackpropagation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gradients II: Backpropagation algorithm

In this practical, we'll investigate the backpropagation algoritithm.  This computes the gradients of the loss with respect to all of the parameters (weights and biases) in the network.  We'll use these gradients when we run stochastic gradient descent.

In [32]:
import numpy as np
import matplotlib.pyplot as plt

First let's define a neural network.  We'll just choose the weights and biaes randomly for now

In [33]:
# Set seed so we always get the same random numbers
np.random.seed(0)

# Number of layers
K = 5
# Number of neurons per layer
D = 6
# Input layer
D_i = 1
# Output layer 
D_o = 1

# Make empty lists 
all_weights = [None] * (K+1)
all_biases = [None] * (K+1)

# Create input and output layers
all_weights[0] = np.random.normal(size=(D, D_i))
all_weights[-1] = np.random.normal(size=(D_o, D))
all_biases[0] = np.random.normal(size =(D,1))
all_biases[-1]= np.random.normal(size =(D_o,1))

# Create intermediate layers
for layer in range(1,K):
  all_weights[layer] = np.random.normal(size=(D,D))
  all_biases[layer] = np.random.normal(size=(D,1))  

#print(all_weights)
#print("")
#print(all_biases)

In [34]:
# Define the Rectified Linear Unit (ReLU) function
def ReLU(preactivation):
  activation = preactivation.clip(0.0)
  return activation

Now let's run our random network.  The weight matrices $\boldsymbol\Omega_{1\ldots K}$ are the entries of the list "all_weights" and the biases $\boldsymbol\beta_{1\ldots k}$ are the entries of the list "all_biases"

We know that we will need the activations $\mathbf{f}_{0\ldots K}$ and the activations $\mathbf{h}_{1\ldots K}$ for the forward pass of backpropagation, so we'll store and return these as well. 


In [35]:
def compute_network_output(net_input, all_weights, all_biases):

  # Retrieve number of layers
  K = len(all_weights) -1

  # We'll store the pre-activations at each layer in a list "all_f"
  # and the activations in a second list[all_h].  
  all_f = [None] * (K+1)
  all_h = [None] * (K+1)

  #For convenience, we'll set 
  # all_h[0] to be the input, and all_f[K] will be the output
  all_h[0] = net_input

  # Run through the layers, calculating all_f[0...K-1] and all_h[1...K]
  for layer in range(K):
      # Update preactivations and activations at this layer according to eqn 7.5
      # Remmember to use np.matmul for matrix multiplications
      # TODO -- Replace the lines below
      all_f[layer] = all_biases[layer] + np.matmul(all_weights[layer],all_h[layer])
      all_h[layer+1] = ReLU(all_f[layer])

  # Compute the output from the last hidden layer
  # TO DO -- Replace the line below
  all_f[K] = all_biases[K]+np.matmul(all_weights[K],all_h[K])

  # Retrieve the output
  net_output = all_f[K]

  return net_output, all_f, all_h

In [36]:
# Define in input
net_input = np.ones((D_i,1)) * 1.2
# Compute network output
net_output, all_f, all_h = compute_network_output(net_input,all_weights, all_biases)
print("True output = %3.3f, Your answer = %3.3f"%(1.907, net_output[0,0]))

True output = 1.907, Your answer = 1.907


Now let's define a loss function.  We'll just use the least squaures loss function. We'll also write a function to compute dloss_doutpu

In [37]:
def least_squares_loss(net_output, y):
  return np.sum((net_output-y) * (net_output-y))

def d_loss_d_output(net_output, y):
    return 2*(net_output -y); 

In [38]:
y = np.ones((D_o,1)) * 20.0
loss = least_squares_loss(net_output, y)
print("y = %3.3f Loss = %3.3f"%(y, loss))

y = 20.000 Loss = 327.371


Now let's compute the derivatives of the network.  We already computed the forward pass.  Let's compute the backward pass.

In [120]:
# We'll need the indicator function
def indicator_function(x):
  x_in = np.array(x)
  x_in[x_in>=0] = 1
  x_in[x_in<0] = 0
  return x_in

# Main backward pass routine
def backward_pass(all_weights, all_biases, all_f, all_h, y):
  # We'll store the derivatives dl_dweights and dl_dbiases in lists as well
  all_dl_dweights = [None] * (K+1)
  all_dl_dbiases = [None] * (K+1)
  # And we'll store the derivatives of the loss with respect to the activation and preactivations in lists
  all_dl_df = [None] * (K+1)
  all_dl_dh = [None] * (K+1)
  # Again for convenience we'll stick with the convention that all_h[0] is the net input and all_f[k] in the net output

  # Compute derivatives of net output with respect to loss
  all_dl_df[K] = np.array(d_loss_d_output(all_f[K],y))

  # Now work backwards through the network
  for layer in range(K,-1,-1):
    # TODO Calculate the derivatives of biases at layer from all_dl_df[K]. (eq 7.13, line 1)
    # NOTE!  To take a copy of matrix X, use Z=np.array(X)
    # REPLACE THIS LINE
    all_dl_dbiases[layer] = np.array(all_dl_df[layer])

    # TODO Calculate the derivatives of weight at layer from all_dl_df[K] and all_h[K] (eq 7.13, line 2)
    # Don't forget to use np.matmul
    # REPLACE THIS LINE
    #print(np.transpose(np.array(all_h[layer])))
    #print(np.array(all_dl_df[layer]))
    #print("--------------------------------")
    all_dl_dweights[layer] = np.matmul(np.array(all_dl_df[layer]),np.transpose(np.array(all_h[layer])))

    # TODO: calculate the derivatives of activations from weight and derivatives of next preactivations (eq 7.13, line 3 second part)
    # REPLACE THIS LINE
    all_dl_dh[layer] = np.matmul(np.transpose(np.array(all_weights[layer])),np.array(all_dl_df[layer]))

    if layer > 0:
      # TODO Calculate the derivatives of the pre-activation f with respect to activation h (eq 7.13, line 3, first part)
      # REPLACE THIS LINE
      #print(np.array(list(map(lambda f: 1 if f > 0 else 0, all_f[layer-1]))))
      #print(np.transpose(np.array(all_dl_dh[layer])))
      #print("--------------------------------")
      all_dl_df[layer-1] = np.transpose(np.array(list(map(lambda f: 1 if f > 0 else 0, np.array(all_f[layer-1])))) * np.transpose(np.array(all_dl_dh[layer])))
      #print(np.transpose(all_dl_df))

  return all_dl_dweights, all_dl_dbiases

In [121]:

#print("--------------------------------")
#print(np.array(all_h))
#print("--------------------------------")
#print(np.array(all_f))
all_dl_dweights, all_dl_dbiases = backward_pass(all_weights, all_biases, all_f, all_h, y)

In [122]:
np.set_printoptions(precision=3)
# Make space for derivatives computed by finite differences
all_dl_dweights_fd = [None] * (K+1)
all_dl_dbiases_fd = [None] * (K+1)

# Let's test if we have the derivatives right using finite differences
delta_fd = 0.000001

# For every layer
for layer in range(K):
  dl_dbias  = np.zeros_like(all_dl_dbiases[layer])
  # For every element in the bias
  for row in range(all_biases[layer].shape[0]):
    # Take copy of biases  We'll change one element each time
    all_biases_copy = [np.array(x) for x in all_biases]
    all_biases_copy[layer][row] += delta_fd
    network_output_1, *_ = compute_network_output(net_input, all_weights, all_biases_copy)
    network_output_2, *_ = compute_network_output(net_input, all_weights, all_biases)
    dl_dbias[row] = (least_squares_loss(network_output_1, y) - least_squares_loss(network_output_2,y))/delta_fd
  all_dl_dbiases_fd[layer] = np.array(dl_dbias)
  print("Bias %d, derivatives from backprop:"%(layer))
  print(all_dl_dbiases[layer])
  print("Bias %d, derivatives from finite differences"%(layer))
  print(all_dl_dbiases_fd[layer])


# For every layer
for layer in range(K):
  dl_dweight  = np.zeros_like(all_dl_dweights[layer])
  # For every element in the bias
  for row in range(all_weights[layer].shape[0]):
    for col in range(all_weights[layer].shape[1]):
      # Take copy of biases  We'll change one element each time
      all_weights_copy = [np.array(x) for x in all_weights]
      all_weights_copy[layer][row][col] += delta_fd
      network_output_1, *_ = compute_network_output(net_input, all_weights_copy, all_biases)
      network_output_2, *_ = compute_network_output(net_input, all_weights, all_biases)
      dl_dweight[row][col] = (least_squares_loss(network_output_1, y) - least_squares_loss(network_output_2,y))/delta_fd
  all_dl_dweights_fd[layer] = np.array(dl_dweight)
  print("Weight %d, derivatives from backprop:"%(layer))
  print(all_dl_dweights[layer])
  print("Weight %d, derivatives from finite differences"%(layer))
  print(all_dl_dweights_fd[layer])

Bias 0, derivatives from backprop:
[[ -4.486]
 [  4.947]
 [  6.812]
 [ -3.883]
 [-24.935]
 [  0.   ]]
Bias 0, derivatives from finite differences
[[ -4.486]
 [  4.947]
 [  6.812]
 [ -3.883]
 [-24.935]
 [  0.   ]]
Bias 1, derivatives from backprop:
[[ -0.   ]
 [-11.297]
 [  0.   ]
 [  0.   ]
 [-10.722]
 [  0.   ]]
Bias 1, derivatives from finite differences
[[  0.   ]
 [-11.297]
 [  0.   ]
 [  0.   ]
 [-10.722]
 [  0.   ]]
Bias 2, derivatives from backprop:
[[-0.   ]
 [-0.   ]
 [ 0.938]
 [ 0.   ]
 [-9.993]
 [ 0.508]]
Bias 2, derivatives from finite differences
[[ 0.   ]
 [ 0.   ]
 [ 0.938]
 [ 0.   ]
 [-9.993]
 [ 0.508]]
Bias 3, derivatives from backprop:
[[-0.   ]
 [-4.8  ]
 [-1.661]
 [-0.   ]
 [ 3.393]
 [ 5.391]]
Bias 3, derivatives from finite differences
[[ 0.   ]
 [-4.8  ]
 [-1.661]
 [ 0.   ]
 [ 3.393]
 [ 5.391]]
Bias 4, derivatives from backprop:
[[-0.   ]
 [ 0.   ]
 [ 0.   ]
 [-0.   ]
 [-5.212]
 [-0.   ]]
Bias 4, derivatives from finite differences
[[ 0.   ]
 [ 0.   ]
 [ 0.   ]
 [